In [ ]:
# Import libraries

import sys
import pypsa
import logging
import warnings
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from plotly import express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.demand import projected_energy
from library.weather import generate_cutout
from library.renewables import capacity_factor, availability_matrix

logging.basicConfig(level=logging.INFO)

In [ ]:
# Set the configuration

## Parameters you won't change very often
base_year = 2024
base_currency = 'SEK'
discount_rate = 0.04
exchange_rates = {
    'EUR': 11.68
}
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2035
use_offwind = False
use_h2 = True
use_nuclear = False
biogas_production_max_nominal = 0

# Legacy config object
config = {
    "base-currency": "SEK",
    "exchange-rates": {
        "EUR": 11.68
    },
    "onwind_turbine": "2030_5MW_onshore.yaml",
    "offwind_turbine": "2030_20MW_offshore.yaml",
    "weather": 2023,
    "base-year": 2024,
    "target-year": 2035, 
    "costs": 2030,
    "discount-rate": 0.04,
    "demand": 2023,
    "load-target": 19,
    "network": {
        "nuclear": False,
        "h2": True,
        "offwind": False,
        "biogas": "Ingen",
        "onwind-limit": "",
        "offwind-limit": ""
    },
    "geography": "14"
}


In [ ]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
# target_load = projected_energy(target_year, 1.21265) * normalized_demand['se3'].values.flatten() * 1_000_000
target_load = 19 * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = config['geography']
cutout, selections, eez, index = generate_cutout(geo, None, '2023-01', '2023-12')
selection = selections[geo]

capacity_factor_solar = capacity_factor(cutout, selection, 'solar', '', geo, None, '2023-01', '2023-12').values.flatten()
capacity_factor_onwind = capacity_factor(cutout, selection, 'onwind', onwind_turbine, geo, None, '2023-01', '2023-12').values.flatten()
capacity_factor_offwind = capacity_factor(cutout, selection, 'offwind', offwind_turbine, geo, None, '2023-01', '2023-12').values.flatten()


In [ ]:
# Build the network

def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

def annualized_capex(asset):
    return (annuity(float(assumptions.loc[('general', 'discount_rate'), 'value']), float(assumptions.loc[(asset, 'lifetime'), 'value'])) + float(assumptions.loc[(asset, 'FOM'), 'value'])) * float(assumptions.loc[(asset, 'capital_cost'), 'value'])

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = resolution

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    'mixedgas',
    'backstop',
    'nuclear',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown', 'brown', 'white', 'mintgreen']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Load bus location
minx, miny, maxx, maxy = selection.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'Load bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'Solar', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'Onwind', x=midx+0.5, y=midy-0.15)
network.add('Bus', 'Offwind', x=midx-1.25, y=midy-0.75)
network.add('Bus', 'Battery bus', carrier='li-ion', x=midx-0.5, y=midy)
network.add('Bus', 'Battery storage', carrier='li-ion', x=midx-0.5, y=midy)
network.add('Bus', 'Gas turbine', x=midx, y=midy+0.5)
network.add('Bus', 'H2 bus', carrier='h2', x=midx-0.5, y=midy+0.5)
network.add('Bus', 'H2 storage', carrier='h2', x=midx-0.5, y=midy+0.5)
network.add('Bus', 'Biogas market', x=midx, y=midy+0.9)
network.add('Bus', 'Nuclear', carrier='nuclear', x=midx, y=midy+0.9)


## Add loads
network.add('Load', 'Desired load', bus='Load bus',
            p_set=target_load
            )

network.add('Generator', 'Backstop', carrier='backstop', bus='Load bus',
            p_nom_extendable=True,
            capital_cost=assumptions.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=assumptions.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=assumptions.loc[('backstop', 'lifetime'), 'value'],
            )

## Add generators

### Solar
network.add('Generator', 'Solar park', carrier='solar', bus='Solar',
            p_nom_extendable=True, 
            p_max_pu=capacity_factor_solar,
            p_nom_mod=assumptions.loc['solar','unit_size'].value,
            capital_cost= annualized_capex('solar'),
            marginal_cost=assumptions.loc[('solar', 'VOM'), 'value'],
            lifetime=assumptions.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Link', 'Solar load link', carrier='solar', bus0='Solar', bus1='Load bus',
            p_nom_extendable=True,
            )

network.add('Link', 'Solar battery link', carrier='solar', bus0='Solar', bus1='Battery bus',
            p_nom_extendable=True,
            )

network.add('Link', 'Solar H2 link', carrier='solar', bus0='Solar', bus1='H2 bus',
            p_nom_extendable=True,
            )

### Onwind
network.add('Generator', 'Onwind park', carrier='onwind', bus='Onwind',
            p_nom_extendable=True,
            p_max_pu=capacity_factor_onwind,
            p_nom_mod=assumptions.loc['onwind','unit_size'].value,
            capital_cost= annualized_capex('onwind'),
            marginal_cost=assumptions.loc[('onwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['onwind','lifetime'].value,
            )

network.add('Link', 'Onwind load link', carrier='onwind', bus0='Onwind', bus1='Load bus',
            p_nom_extendable=True,
            )

network.add('Link', 'Onwind battery link', carrier='onwind', bus0='Onwind', bus1='Battery bus',
            p_nom_extendable=True,
            )

network.add('Link', 'Onwind H2 link', carrier='onwind', bus0='Onwind', bus1='H2 bus',
            p_nom_extendable=True,
            )

### Offwind
network.add('Generator', 'Offwind park', carrier='offwind', bus='Offwind',
            p_nom_extendable=use_offwind,
            p_max_pu=(capacity_factor_offwind if use_offwind else [0] * len(capacity_factor_offwind)),
            p_nom_mod=assumptions.loc['offwind','unit_size'].value,
            capital_cost= annualized_capex('offwind'),
            marginal_cost=assumptions.loc[('offwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['offwind','lifetime'].value,
            )

network.add('Link', 'Offwind load link', carrier='offwind', bus0='Offwind', bus1='Load bus',
            p_nom_extendable=use_offwind,
            )

network.add('Link', 'Offwind battery link', carrier='offwind', bus0='Offwind', bus1='Battery bus',
            p_nom_extendable=use_offwind,
            )

network.add('Link', 'Offwind H2 link', carrier='offwind', bus0='Offwind', bus1='H2 bus',
            p_nom_extendable=use_offwind,
            )

## Add H2 electrolysis, storage, pipline to gas turbine

network.add('Link', 'toH2', bus0='Load bus', bus1='H2 bus',
            p_nom_extendable=True
            )

network.add('Link', 'H2 electrolysis', carrier='h2', bus0='H2 bus', bus1='H2 storage',
            p_nom_extendable=True,
            p_nom_mod=assumptions.loc['h2_electrolysis','unit_size'].value,
            capital_cost= annualized_capex('h2_electrolysis'),
            marginal_cost=assumptions.loc[('h2_electrolysis', 'VOM'), 'value'],
            lifetime=assumptions.loc['h2_electrolysis','lifetime'].value,
            efficiency=assumptions.loc['h2_electrolysis','efficiency'].value,
            )

network.add('Store', 'H2 storage', carrier='h2', bus='H2 storage',
            e_initial=(150_000 if use_h2 else 0),
            e_nom_extendable=use_h2,
            e_cyclic=True,
            capital_cost= annualized_capex('h2_storage'),
            marginal_cost=assumptions.loc['h2_storage','VOM'].value,
            lifetime=assumptions.loc['h2_storage','lifetime'].value
            )

network.add('Link', 'H2 pipeline', carrier='h2', bus0='H2 storage', bus1='Gas turbine',
            p_nom_extendable=True,
            )

### Biogas pipeline

network.add('Generator', 'Biogas input', carrier='biogas', bus='Biogas market',
            p_nom_extendable=True,
            p_nom_max=biogas_production_max_nominal,
            marginal_cost=assumptions.loc['biogas','cost'].value,
            lifetime=100,
            )

network.add('Link', 'Biogas pipeline', carrier='biogas', bus0='Biogas market', bus1='Gas turbine',
            p_nom_extendable=True,
            )

### Gas turbines
network.add('Link', 'Simple Cycle Gas turbine', carrier='mixedgas', bus0='Gas turbine', bus1='Load bus',
            p_nom_extendable=True,
            p_nom_mod=assumptions.loc['simple_cycle_gas_turbine','unit_size'].value,
            capital_cost= annualized_capex('simple_cycle_gas_turbine'),
            marginal_cost=assumptions.loc['simple_cycle_gas_turbine','VOM'].value,
            lifetime=assumptions.loc['simple_cycle_gas_turbine','lifetime'].value,
            efficiency=assumptions.loc['simple_cycle_gas_turbine','efficiency'].value,
            )

network.add('Link', 'Combined Cycle Gas turbine', carrier='mixedgas', bus0='Gas turbine', bus1='Load bus',
            p_nom_extendable=True,
            p_nom_mod=assumptions.loc['combined_cycle_gas_turbine','unit_size'].value,
            capital_cost= annualized_capex('combined_cycle_gas_turbine'),
            marginal_cost=assumptions.loc['combined_cycle_gas_turbine','VOM'].value,
            lifetime=assumptions.loc['combined_cycle_gas_turbine','lifetime'].value,
            efficiency=assumptions.loc['combined_cycle_gas_turbine','efficiency'].value,
            )

## Add battery storage

network.add('Link', 'toBattery', carrier='li-ion', bus0='Load bus', bus1='Battery bus',
            p_nom_extendable=True
            )

network.add('Link','Battery charge', bus0 = 'Battery bus', bus1 = 'Battery storage',
            p_nom_extendable = True,
            capital_cost= annualized_capex('battery_inverter'),
            marginal_cost=assumptions.loc['battery_inverter','VOM'].value,
            lifetime=assumptions.loc['battery_inverter','lifetime'].value,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

network.add('Store', 'Battery storage', carrier='li-ion', bus='Battery storage',
            e_initial=100,
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost= annualized_capex('battery_storage'),
            marginal_cost=assumptions.loc['battery_storage','VOM'].value,
            lifetime=assumptions.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','Battery discharge', carrier='li-ion', bus0 = 'Battery storage', bus1 = 'Load bus',
            p_nom_extendable = True,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

## Nuclear
num_nuclear_conv = 0 if use_nuclear else 0
num_nuclear_smr = 2 if use_nuclear else 0

#       for i in range(num_nuclear_conv):
network.add('Generator', f"{'Conventional nuclear'}", carrier='nuclear', bus='Nuclear',
            p_nom_extendable=True,
            p_nom_mod=assumptions.loc['nuclear_conv','p_nom'].value,
            p_nom_max=num_nuclear_conv * float(assumptions.loc['nuclear_conv','p_nom'].value),
#                p_nom=assumptions.loc['nuclear_conv','p_nom'].value,
            capital_cost= annualized_capex('nuclear_conv'),
            marginal_cost=float(assumptions.loc['nuclear_conv','VOM'].value) + float(assumptions.loc['nuclear_conv','fuel'].value),
            lifetime=assumptions.loc['nuclear_conv','lifetime'].value,
            )

#        for j in range(num_nuclear_smr):
network.add('Generator', f"{'SMR nuclear'}", carrier='nuclear', bus='Nuclear',
            p_nom_extendable=True,
            p_nom_mod=assumptions.loc['nuclear_smr','p_nom'].value,
            p_nom_max=num_nuclear_smr * float(assumptions.loc['nuclear_smr','p_nom'].value),
#                p_nom=assumptions.loc['nuclear_smr','p_nom'].value,
            capital_cost= annualized_capex('nuclear_smr'),
            marginal_cost=float(assumptions.loc['nuclear_smr','VOM'].value) + float(assumptions.loc['nuclear_smr','fuel'].value),
            lifetime=assumptions.loc['nuclear_smr','lifetime'].value,
            )

network.add('Link', 'Nuclear to load', carrier='nuclear', bus0='Nuclear', bus1='Load bus',
            p_nom_extendable=True,
            )

In [ ]:
network = pypsa.Network()
network.import_from_netcdf(paths.output_path / 'geography=14,target-year=2035,floor=0.5,load-target=19,h2=True,offwind=True,biogas-limit=500/network.nc')
                                               


In [ ]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if use_offwind:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['Offwind park'] - generator_capacity.loc['Onwind park']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["Battery charge"] - network.links.at["Battery charge", "efficiency"] * link_capacity.loc["Battery discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

In [ ]:
network.statistics()

In [ ]:
# Plot the dashboard
bus_colors = ["black", "yellow", "blue", "green", "lightblue", "red", "red", "red", "red"]
network = NETWORK

## Parameters
window_size=56 # Rolling average window size for charts

## Create solar availability map widget
solar_map = widgets.Output()
with solar_map:
    solar_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Reds" })
    plt.show()

## Create onwind availability map widget
onwind_map = widgets.Output()
with onwind_map:
    onwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Greens" })
    plt.show()

## Create offwind availability map widget
offwind_map = widgets.Output()
with offwind_map:
    offwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Blues" })
    plt.show()

## Create solar capacity factor line chart widget
solar_capacity_line_chart = widgets.Output()
with solar_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_SOLAR.plot(ax=ax, color="Red", alpha=0.2)
    AVAIL_CAPACITY_SOLAR.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Red")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create onwind capacity factor line chart widget
onwind_capacity_line_chart = widgets.Output()
with onwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_ONWIND.plot(ax=ax, color="Green", alpha=0.2)
    AVAIL_CAPACITY_ONWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Green")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create offwind capacity factor line chart widget
offwind_capacity_line_chart = widgets.Output()
with offwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_OFFWIND.plot(ax=ax, color="Blue", alpha=0.2)
    AVAIL_CAPACITY_OFFWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Blue")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create target load line chart widget
load_line_chart = widgets.Output()
with load_line_chart:
    fig, ax = plt.subplots(figsize=(14, 7))
    DEMAND.plot(ax=ax, color='grey', alpha=0.2)
    DEMAND.rolling(window=window_size, center=True).mean().plot(ax=ax, color='blue', alpha=1)
    ax.legend(['Demand (3h average)', 'Demand (weekly average'])
    ax.set_ylabel("Load [MW]")
    ax.set_xlabel('')
    plt.show()


## Create network map widget
minx, miny, maxx, maxy = SELECTION.total_bounds
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
with network_map:
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    network.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                            markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
    plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)
    plt.show()

## Format and create the parameters table widget
parameters_table = widgets.HTML(value=parameters.to_html())

## Format and create the system table widget widget
generator_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'p_nom_opt': 'Optimal Capacity'
}

store_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'e_nom_opt': 'Optimal Capacity'
}

generators = pd.concat([
    network.generators.loc[['Solar park', 'Onwind park', 'Offwind park']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']],
    network.links.loc[['Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
])

generators['capital_cost'] = generators['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000, 1)} MSEK/MW")
generators['marginal_cost'] = generators['marginal_cost'].apply(lambda x: f"{round(float(x), 1)} SEK/MWh")
generators['lifetime'] = generators['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
generators['p_nom_opt'] = generators['p_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MW")
generators.rename(columns=generator_column_names, inplace=True)

stores = network.stores.loc[["H2 storage", "Battery storage"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores.loc[:, 'capital_cost'] = stores['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000,1)} MSEK/MWh")
stores.loc[:, 'marginal_cost'] = stores['marginal_cost'].apply(lambda x: f"{round(float(x),1)} SEK/MWh")
stores.loc[:, 'lifetime'] = stores['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
stores.loc[:, 'e_nom_opt'] = stores['e_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MWh")
stores.rename(columns=store_column_names, inplace=True)

system_table = widgets.HTML(value=pd.concat([generators, stores]).to_html())


## Format and create the stores table widget widget

## Create generators size pie chart widget
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators['p_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Generator sizes')
    plt.show()

## Create stores size pie chart widget
store_pie_chart = widgets.Output()
with store_pie_chart:
    network.stores.loc[["H2 storage", "Battery storage"]]['e_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Energy store sizes')
    plt.axis('off')  # Turns off the axis
    plt.show()

## Create price line chart widget
price_line_chart = widgets.Output()
with price_line_chart:
    network.buses_t.marginal_price.plot(figsize=(24, 6), legend=True, title='Marginal price')
    plt.show()

## Create generators outpuExclusionContainert line chart widget
generator_line_chart = widgets.Output()
with generator_line_chart:
    fig, ax = plt.subplots(figsize=(16,6))
    network.generators_t.p.abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
    network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)
    plt.show()

## Create generators total output pie chart
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators_t.p.sum().plot(figsize=(8, 6), kind='pie', labels=None, legend=True, title='Power output')
    plt.show()


## Create stores line chart widget
store_line_chart = widgets.Output()
with store_line_chart:
    network.stores_t.e[['H2 storage', 'Battery storage']].plot(figsize=(24, 6),legend=True, title='Energy stored')
    plt.show()


## Build the dashboard and show

sun_wind_map = widgets.HBox([solar_map, onwind_map, offwind_map])
sun_wind_cfs = widgets.VBox([solar_capacity_line_chart, onwind_capacity_line_chart, offwind_capacity_line_chart])

row_1 = widgets.HBox([widgets.VBox([load_line_chart, sun_wind_map]), sun_wind_cfs])

row_2 = widgets.HBox([widgets.VBox([network_map, system_table, widgets.HBox([generator_pie_chart, store_pie_chart])]), parameters_table])

row_3 = widgets.HBox([generator_line_chart, generator_pie_chart])

dashboard = widgets.VBox([row_1, row_2, row_3, store_line_chart, price_line_chart])

dashboard


In [ ]:
bus_colors = ["black", "yellow", "blue", "green", "lightblue", "red", "red", "red", "red", "black"]

minx, miny, maxx, maxy = SELECTION.total_bounds
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
NETWORK.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                        markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)


In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
NETWORK.buses_t.marginal_price[['Load bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
NETWORK.stores_t.e[['H2 storage']].plot(ax=ax2,legend=True, color='red', title='Energy stored')


In [ ]:
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')

fix, ax1 = plt.subplots(figsize=(24, 6))
                        
NETWORK.buses_t.marginal_price[['Load bus']].loc[start_date:end_date].plot(ax=ax1, legend=True, title='Marginal price')

#ax2 = plt.twinx(ax1)
#DEMAND.plot(ax=ax2, color='grey', alpha=0.2)

#NETWORK.buses_t.marginal_price.loc[start_date:end_date].mean()


In [ ]:
total_cost = NETWORK.statistics.capex().sum()
total_energy = NETWORK.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
network.statistics.capex()

In [ ]:
NETWORK.generators_t.p.abs().rolling(window=56, center=True).mean().plot(figsize=(16,6), kind='area', stacked=True)

In [ ]:
NETWORK.generators_t.p.abs().plot(figsize=(16,6), kind='area', stacked=True)

In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))                   
DEMAND.plot(ax=ax1, color='grey', alpha=0.2)

In [ ]:
column_names = {
    'annual_cost': 'Årskostnad',
    'lifetime': 'Livslängd',
    'p_nom_opt': 'Kapacitet',
    'generators': 'Kraftverk',
    'energy_produced': 'Producerad energi',
}

generator_index = NETWORK.generators.index.difference(['Backstop', 'Biogas input'])

generators = pd.concat([
    NETWORK.generators.loc[NETWORK.generators.index.isin(generator_index)][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt', 'p_nom_mod']],
    NETWORK.links.loc[['Combined Cycle Gas turbine', 'Simple Cycle Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt', 'p_nom_mod']]
])

generators['annual_cost'] = np.where(generators['p_nom_opt'] != 0, generators['capital_cost'] * generators['p_nom_opt'], 0)
generators['energy_produced'] = NETWORK.generators_t.p.loc[:, generator_index].sum() * 3
generators['generators'] = generators['p_nom_opt'] / generators['p_nom_mod']

other = NETWORK.links.loc[['H2 electrolysis', 'Battery charge']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
other['annual_cost'] = np.where(other['p_nom_opt'] != 0, other['capital_cost'] * other['p_nom_opt'], 0)

stores = NETWORK.stores.loc[["H2 storage", "Battery storage"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores['annual_cost'] = np.where(stores['e_nom_opt'] != 0, stores['capital_cost'] * stores['e_nom_opt'], 0)
stores.rename(columns={'e_nom_opt': 'p_nom_opt'}, inplace=True)

biogas = NETWORK.generators.loc[['Biogas input']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
biogas['energy_produced'] = NETWORK.generators_t.p[['Biogas input']].sum() * 3 * float(parameters.loc['biogas', 'efficiency'].value)
biogas['annual_cost'] = np.where(biogas['p_nom_opt'] != 0, biogas['marginal_cost'] * biogas['energy_produced'] / float(parameters.loc['biogas', 'efficiency'].value), 0)
biogas[['lifetime', 'p_nom_opt', 'total_cost']] = 0

system = pd.concat([generators, other, stores, biogas]).drop(columns=['capital_cost', 'marginal_cost'])[['p_nom_opt', 'generators', 'lifetime', 'energy_produced', 'annual_cost']]

totals = pd.DataFrame(columns=system.columns, index=['Total'])
totals['energy_produced'] = system['energy_produced'].sum()
totals['annual_cost'] = system['annual_cost'].sum()

system = pd.concat([system, totals]).rename(columns=column_names)

years = lambda s: f'{s:,.0f} år'.replace(',', ' ')
million_cost = lambda s: f'{s/1_000_000:,.0f} MSEK'.replace(',', ' ')
energy = lambda s: f'{s/1_000:,.0f} GWh'.replace(',', ' ')
price = lambda s: f'{s:,.0f} SEK'.replace(',', ' ')
system.abs().style.format({'Livslängd': years, 'Producerad energi': energy, 'Pris': price, 'Årskostnad': million_cost, 'Totalkostnad': million_cost}, precision=0, na_rep='-')

In [ ]:
f"{system['Årskostnad']['Total'] / (NETWORK.loads_t.p['Desired load'].sum() * 3) / 1000 * 100:.1f} öre/kWh"

In [ ]:
start_date = pd.to_datetime('2023-08-01')
end_date = pd.to_datetime('2023-09-01')
fig1, ax1 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Solar park']].loc[start_date:end_date].plot(ax=ax1, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']].loc[start_date:end_date].plot(ax=ax1, kind='area', stacked=True, legend=True, alpha=0.2)

fig2, ax2 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Onwind park']].loc[start_date:end_date].plot(ax=ax2, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']].loc[start_date:end_date].plot(ax=ax2, kind='area', stacked=True, legend=True, alpha=0.2)

fig3, ax3 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Offwind park']].loc[start_date:end_date].plot(ax=ax3, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Offwind battery link', 'Offwind H2 link', 'Offwind load link']].loc[start_date:end_date].plot(ax=ax3, kind='area', stacked=True, legend=True, alpha=0.2)

In [ ]:
solar_curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]
solar_curtailment['Curtailment'] = solar_curtailment['Solar park']
solar_curtailment.drop(columns=['Solar park'], inplace=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 6))

pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

wind_curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
wind_curtailment['Curtailment'] = wind_curtailment['Onwind park']
wind_curtailment.drop(columns=['Onwind park'], inplace=True)

pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], wind_curtailment]).resample('ME').sum().plot(ax=ax2, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
curtailment['Curtailment'] = curtailment['Onwind park']
curtailment.drop(columns=['Onwind park'], inplace=True)

fig1, ax1 = plt.subplots(figsize=(24, 6))
pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
NETWORK.loads_t.p[['Desired load']].sum()*3

In [ ]:
curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]

curtailment.plot(figsize=(24, 6), alpha=0.2)


In [ ]:
fig, ax = plt.subplots(figsize=(24,8))
network.generators_t.p[['Solar park']].abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
#network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)

In [ ]:
fig, ax = plt.subplots(figsize=(24,8))
network.stores_t.e[['Battery storage']].abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')